In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

from addition_data import *
import sys
sys.path.insert(0,'../../')
import copy

#from texar.modules import TransformerEncoder, TransformerDecoder

#from rcalgo.tf.tfmodels.seq2seq import *
#import texar as tx
from collections import Counter
import random
import warnings
warnings.filterwarnings('ignore')

from rcalgo.tf.tfmodels.texar_models.tx_transformer_seq2seq import *
from rcalgo.tf.utils.text_utils import *
#from texar.utils import transformer_utils


In [2]:
DIGITS = 3
SAMPLE_SIZE = 50000
nwords = 0
maxlen = DIGITS*2 + 1 + 2
target_maxlen = DIGITS + 1 #add eos
x, y = generate_addtion_pairs(sample_size=SAMPLE_SIZE, digits=DIGITS, raw_text=True)
#x_seq = np.sum(x > 0, axis=1) #np.zeros(shape=(SAMPLE_SIZE)) + maxlen
#y_seq = np.sum(y > 0, axis=1) + 1

Generating data...
Total addition questions: 50000


In [3]:
word_freq = word_count(x, word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=1)
id_term_map = dict([(v, k) for k,v in term_id_map.items()])

In [4]:
from rcalgo.tf.utils.train_config import *

args = parse_initial_training_args(
    [
        '--learning-rate', '0.0001',
       #'--nb-words', str(nwords),
        '--oov-buckets', '1',
        '--n-epochs', '10',
        '--opt', 'adam',
        '--batch-size', '64',
        '--p-coef', '0',
        '--n-head', '8',
        '--position-type','2,3,4,5',
        '--transformer-func','relu',
        '--keep-prob','1.0',
        '--embedding-size', '256',
        '--hidden-size', '256',
        '--gpu-id', '0',
        '--gpu-num', '1',
        '--expand-input',
        '--num-layers', '1',
        '--position-info',
        '--maxlen', str(maxlen)
    ]
)
args.output_maxlen = target_maxlen
args.beam_width = 2

In [5]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
seq2seq_model = TxTransformerSeq2SeqModel(args, graph_to_use, word_dict=term_id_map)
seq2seq_model.build_model()
seq2seq_model.build_model_summary()
display(seq2seq_model.model_summary())
#seq2seq_model.run([x,y], test_size=0.2)


total parameter number: 2770944


variable_name  variable_shape  \
0     gpu/embedding/embedding_layer/embedding_table:0       [14, 256]   
1            gpu/mw_conv_block/conv_1d/he_uniform/W:0   [2, 256, 256]   
2                       gpu/mw_conv_block/conv_1d/B:0           [256]   
3          gpu/mw_conv_block/conv_1d_1/he_uniform/W:0   [3, 256, 256]   
4                     gpu/mw_conv_block/conv_1d_1/B:0           [256]   
5          gpu/mw_conv_block/conv_1d_2/he_uniform/W:0   [4, 256, 256]   
6                     gpu/mw_conv_block/conv_1d_2/B:0           [256]   
7          gpu/mw_conv_block/conv_1d_3/he_uniform/W:0   [5, 256, 256]   
8                     gpu/mw_conv_block/conv_1d_3/B:0           [256]   
9                 gpu/pos_embedding/embedding_table:0        [9, 256]   
10  gpu/transform2_block/multi_head_attention/conv...   [1, 256, 768]   
11  gpu/transform2_block/multi_head_attention/conv...           [768]   
12  gpu/transform2_block/multi_head_attention/conv...   [1, 256, 256]   
13  gpu/transform2_block/multi_head_attention/conv...           [256]   
14               gpu/transform2_block/ln_layer/beta:0           [256]   
15              gpu/transform2_block/ln_layer/gamma:0           [256]   
16  gpu/transform2_block/ffn_mlp/conv_1d/he_unifor...  [1, 256, 1024]   
17           gpu/transform2_block/ffn_mlp/conv_1d/B:0          [1024]   
18  gpu/transform2_block/ffn_mlp/conv_1d_1/he_unif...  [1, 1024, 256]   
19         gpu/transform2_block/ffn_mlp/conv_1d_1/B:0           [256]   
20             gpu/transform2_block/ln_layer_1/beta:0           [256]   
21            gpu/transform2_block/ln_layer_1/gamma:0           [256]   
22                          gpu/position_embedder/w:0        [4, 256]   
23             gpu/transformer_decoder/layer_0/beta:0           [256]   
24            gpu/transformer_decoder/layer_0/gamma:0           [256]   
25  gpu/transformer_decoder/layer_0/self_attention...      [256, 256]   
26  gpu/transformer_decoder/layer_0/self_attention...      [256, 256]   
27  gpu/transformer_decoder/layer_0/self_attention...      [256, 256]   
28  gpu/transformer_decoder/layer_0/self_attention...      [256, 256]   
29  gpu/transformer_decoder/layer_0/encdec_attenti...           [256]   
30  gpu/transformer_decoder/layer_0/encdec_attenti...           [256]   
31  gpu/transformer_decoder/layer_0/encdec_attenti...      [256, 256]   
32  gpu/transformer_decoder/layer_0/encdec_attenti...      [256, 256]   
33  gpu/transformer_decoder/layer_0/encdec_attenti...      [256, 256]   
34  gpu/transformer_decoder/layer_0/encdec_attenti...      [256, 256]   
35  gpu/transformer_decoder/layer_0/past_poswise_l...           [256]   
36  gpu/transformer_decoder/layer_0/past_poswise_l...           [256]   
37  gpu/transformer_decoder/layer_0/ffn/conv1/kern...     [256, 1024]   
38   gpu/transformer_decoder/layer_0/ffn/conv1/bias:0          [1024]   
39  gpu/transformer_decoder/layer_0/ffn/conv2/kern...     [1024, 256]   
40   gpu/transformer_decoder/layer_0/ffn/conv2/bias:0           [256]   
41                     gpu/transformer_decoder/beta:0           [256]   
42                    gpu/transformer_decoder/gamma:0           [256]   
43             gpu/transformer_decoder/dense/kernel:0       [256, 15]   

   parameters  
0        3584  
1      131072  
2         256  
3      196608  
4         256  
5      262144  
6         256  
7      327680  
8         256  
9        2304  
10     196608  
11        768  
12      65536  
13        256  
14        256  
15        256  
16     262144  
17       1024  
18     262144  
19        256  
20        256  
21        256  
22       1024  
23        256  
24        256  
25      65536  
26      65536  
27      65536  
28      65536  
29        256  
30        256  
31      65536  
32      65536  
33      65536  
34      65536  
35        256  
36        256  
37     262144  
38       1024  
39     262144  
40        256  
41        256  
42        256  
43       3840

In [ ]:
with seq2seq_model.job.session.graph.as_default():
    input_var = tf.get_collection('input_dict')[0]
    outputs = tf.get_collection('output_dict')[0:5]
    logits = tf.get_collection('output_dict')[-1]
    greedy_logits, greedy_sample_id, sample_logits, sample_sample_id, beam_search_results = outputs
    #greedy_logits, greedy_sample_id  = outputs

In [ ]:
def out_print(num_arr, id_term_map, eos=12):
    text = ''
    for i in num_arr:
        if i > eos:
            break
        else:
            text += id_term_map[i]
    return text

In [ ]:
output = seq2seq_model.job.session.run(greedy_sample_id, feed_dict={input_var:x[0:10]})
for i in range(0, len(output)):
    print(x[i], y[i], out_print(output[i], id_term_map))